In [1]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
import seaborn as sns
import pickle

import matplotlib.pyplot as plt
plt.style.use("ggplot")
%config InlineBackend.figure_format='retina'

import QuantTrading.ImpactFitting as IF

def load_from_pickle(filename):
    path = '../pkl_dump/'
    with open(path + filename, 'rb') as f:
        return pickle.load(f)

# Load data
traded_volume_df = load_from_pickle('traded_volume_df.pkl')
px_df = load_from_pickle('px_df.pkl')
daily_stock_info_df = load_from_pickle('daily_stock_info_df.pkl')
monthly_scaling_factor = load_from_pickle('monthly_scaling_factor.pkl')
stocks = traded_volume_df.reset_index()["stock"].unique()

In [2]:
traded_volume_df = traded_volume_df.groupby('date').mean()
px_df = px_df.groupby('date').mean()
monthly_scaling_factor = monthly_scaling_factor.groupby('date').mean()

In [3]:
model_type = "linear"
half_life = 3600
in_sample_month = 3


impact_px_df = IF.get_impact_state(traded_volume_df, monthly_scaling_factor, 
                                half_life, model_type)
reg_summary_temp = IF.get_regression_results(impact_px_df, px_df, 
                                        in_sample_month, explanation_horizon_periods=6, all=True)

In [4]:
reg_summary_temp

,is_xy,is_xx,is_yy,is_x,is_y,is_count,oos_xy,oos_xx,oos_yy,oos_x,oos_y,oos_count,beta_estimate,alpha_estimate,is_sse,is_mse,is_rsq,oos_sse,oos_mse,oos_rsq
0,0.000001,2.408250e-09,0.002501,0.002948,5.220906,15997.0,5.748663e-07,1.115777e-09,0.000993,0.001864,3.135527,13113.0,221.30069,0.000286,0.000797,0.000706,0.114548,0.000243,0.000307,-0.263997
